In [70]:
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [71]:
import re
def filter_tags(htmlstr):
    """
    # Python通过正则表达式去除(过滤)HTML标签
    :param htmlstr:
    :return:
    """
    # 先过滤CDATA
    re_cdata = re.compile('//<!\
    CDATA\[[ >]∗ //\
    CDATA\[[ >]∗ //\
    \] > ',re.I) #匹配CDATA
    re_script = re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>', re.I)
    # Script
    re_style = re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>', re.I)
    # style
    re_br = re.compile('<br\s*?/?>')
    # 处理换行
    re_h = re.compile('</?\w+[^>]*>')
    # HTML标签
    re_comment = re.compile('<!--[^>]*-->')
    # HTML注释
    s = re_cdata.sub('', htmlstr)
    # 去掉CDATA
    s = re_script.sub('', s)  # 去掉SCRIPT
    s = re_style.sub('', s)
    # 去掉style
    s = re_br.sub('\n', s)
    # 将br转换为换行
    s = re_h.sub('', s)  # 去掉HTML 标签
    s = re_comment.sub('', s)
    # 去掉HTML注释
    # 去掉多余的空行
    blank_line = re.compile('\n+')
    s = blank_line.sub('\n', s)
    s = replaceCharEntity(s)  # 替换实体
    return s

def replaceCharEntity(htmlstr):
    """
    :param htmlstr:HTML字符串
    :function:过滤HTML中的标签
    """
    CHAR_ENTITIES = {'nbsp': ' ', '160': ' ',
                     'lt': '<', '60': '<',
                     'gt': '>', '62': '>',
                     'amp': '&', '38': '&',
                     'quot': '"', '34': '"', }

    re_charEntity = re.compile(r'&#?(?P<name>\w+);')
    sz = re_charEntity.search(htmlstr)
    while sz:
        entity = sz.group()  # entity全称，如>
        key = sz.group('name')  # 去除&;后entity,如>为gt
        try:
            htmlstr = re_charEntity.sub(CHAR_ENTITIES[key], htmlstr, 1)
            sz = re_charEntity.search(htmlstr)
        except KeyError:
            # 以空串代替
            htmlstr = re_charEntity.sub('', htmlstr, 1)
            sz = re_charEntity.search(htmlstr)
    return htmlstr

def repalce(s, re_exp, repl_string):
    return re_exp.sub(repl_string,s)

In [72]:
with open("f:/data/complaint1.txt", "r", encoding="GBK", errors='ignore') as f:
    texts = [filter_tags(line).replace('\n', "") for line in f.readlines()]

In [102]:
datasets = []
stopwords = {}.fromkeys([ line.rstrip() for line in open('f:/data/baidu_stopword.txt', 'r', encoding='utf-8') ])

for line in texts:
    segs = [word for word in jieba.lcut(line, cut_all=False) if word not in stopwords]
    datasets.append(segs)


In [103]:
from gensim import corpora
from gensim.models import TfidfModel
dct=corpora.Dictionary(datasets)
corpus = [dct.doc2bow(line) for line in datasets]

In [104]:
model = TfidfModel(corpus)
vector = model[corpus[0]]
# print(vector)
#print(model)


In [105]:
dict = {}
for l in model[corpus]:
    for w in l:
        if w[0] not in dict:
            dict[w[0]] = w[1]

with open("f:/data/complaint_idf.txt.big", "w") as f:
    for (k, v) in dict.items():
        f.write(dct[k])
        f.write(" ")
        f.write(str(v))
        f.write("\n")
